In [1]:
import torch
import torch.nn as nn
import numpy as np
from torch.autograd import Variable

class rf_agent(nn.Module):
    def __init__(self):
        super(rf_agent, self).__init__()
        
        self.layer1=nn.Sequential(nn.Conv2d(1,15,kernel_size=4,
        stride=2,padding=1),
        nn.ReLU(inplace=True))
        
        self.layer2=nn.Sequential(nn.Conv2d(15,30,kernel_size=2,
        stride=2,padding=0),
        nn.ReLU(inplace=True))
        
        self.layer3=nn.LSTM(270, 120)
        
        self.last_layer=nn.Linear(120,4)
        
        self.reset()

        

    def forward(self, x):
        x=self.layer1(x)
        x=self.layer2(x)       
        x = x.view(1,x.size()[0], -1)
        x,self.hidden=self.layer3(x,self.hidden)
        
        x=self.last_layer(x)
        return x
    
    def reset(self):
        self.hidden = (Variable(torch.zeros(1, 1,120)),Variable(torch.zeros(1, 1,120)))
                       
model=rf_agent()

x=torch.tensor(np.random.randn(10,1,15,15)).float() 

for data in x:
    data=data.reshape(1,1,15,15)
    y=model(data)
    print(y)

tensor([[[ 0.0247,  0.0589, -0.0889, -0.0368]]], grad_fn=<AddBackward0>)
tensor([[[ 0.0242,  0.0701, -0.0995, -0.0472]]], grad_fn=<AddBackward0>)
tensor([[[ 0.0282,  0.0415, -0.0946, -0.0309]]], grad_fn=<AddBackward0>)
tensor([[[ 0.0145,  0.0432, -0.0910, -0.0362]]], grad_fn=<AddBackward0>)
tensor([[[ 0.0401,  0.0225, -0.0917, -0.0261]]], grad_fn=<AddBackward0>)
tensor([[[ 0.0282,  0.0451, -0.0869, -0.0201]]], grad_fn=<AddBackward0>)
tensor([[[ 0.0316,  0.0445, -0.1057, -0.0609]]], grad_fn=<AddBackward0>)
tensor([[[ 0.0359,  0.0243, -0.0818, -0.0447]]], grad_fn=<AddBackward0>)
tensor([[[ 0.0173,  0.0338, -0.0979, -0.0541]]], grad_fn=<AddBackward0>)
tensor([[[-0.0123,  0.0766, -0.1015, -0.0729]]], grad_fn=<AddBackward0>)


In [3]:
import pygame
import sys, os
import random
import torch
import torch.nn as nn

import numpy as np
from copy import copy
from pygame.locals import *
from gridworld import grid_world
from policy import Inp, Crew
from rf_network import rf_agent
from utils import im2col
import torch.nn.functional as F
import torch.optim as optim




env=grid_world()
agent_pos1,agent_pos2,agent_pos3,agent_inp_pos=env.reset()
train_dataset=[]
criterion=nn.CrossEntropyLoss()

cr1=Crew()
cr2=Crew()
cr3=Crew()
inp=Inp()
action_inp=0
Start=False
model=rf_agent()
for num in range(3):
    train_data=[]
    train_label=[]
    cycle=0
    while True:
        #env.render()
        for event in pygame.event.get():
            if event.type == KEYDOWN:
                Start=True
        if Start==False:
            continue
        action1=cr1.forward(agent_pos1,agent_inp_pos)
        action2=cr2.forward(agent_pos2,agent_inp_pos)
        action3=cr3.forward(agent_pos3,agent_inp_pos)
        action_inp=inp.forward(agent_inp_pos,agent_pos1,agent_pos2,agent_pos3)
        agent_pos1,agent_pos2,agent_pos3,agent_inp_pos,done,data=env.step(action1,action2,action3,action_inp)
        inp_f=agent_inp_pos[0]//15+3*(agent_inp_pos[1]//15)
        cycle=(cycle+1)%30
        
        
        
        #train_data.append(data)
        #train_label.append([action1,action2,action3,action_inp])
        if cycle==0:
            data=data.reshape(1,1,60,45)
            data=im2col(data,15,15,15)

            
            d=torch.tensor(data[:,inp_f,:,:].reshape(1,1,15,15)).float()
            y=model(d)
            label=torch.tensor(action_inp)
            loss=criterion(y,label)
            print(loss)
            """"if len(train_data)==0:
                train_data=data
                train_label=action_inp
            else:
                train_data=np.vstack([train_data,data])
                train_label=np.vstack([train_label,action_inp])"""
    
        if done==True:
            agent_pos1,agent_pos2,agent_pos3,agent_inp_pos=env.reset()
            model.reset()
            
            print('episode{} finished!'.format(num))
            break




    pygame.display.flip()

ValueError: too many values to unpack (expected 4)

In [13]:
from utils import im2col

d=train_data.reshape(-1,1,60,45)
d=im2col(d,15,15,15)

In [52]:
N = 64
y = torch.empty(N, dtype=torch.long).random_(10)
yPred = torch.randn(N, 10)
yPred

tensor([[ 0.0293, -1.9780, -0.1416, -0.0460,  0.5663, -0.2847, -1.4861,  1.2004,
          0.1414,  0.3929],
        [-0.7073,  0.5140,  1.7092,  0.9273, -0.6131,  1.8087, -1.5484,  0.3158,
          0.4164, -1.0406],
        [-0.3177, -0.2155, -2.4730, -0.4629, -1.7920,  0.4787, -0.4281, -1.5473,
          1.4223, -0.9630],
        [ 1.0059, -1.2744, -0.6129,  0.3236,  1.6436,  0.1316, -0.1441, -0.7278,
          0.4666,  0.4735],
        [-1.3644,  0.5745,  3.0385, -1.5359,  0.6655,  0.4070, -1.2996,  0.2000,
         -1.3929, -0.2883],
        [ 0.2740,  1.3945, -3.0087, -1.1995,  0.6598,  0.4671, -0.9221, -1.3103,
          0.5970,  1.2085],
        [-1.2940,  1.0679, -0.7401, -0.8908, -1.8026,  0.2566, -1.7485, -0.7871,
          2.0037,  0.7893],
        [-1.1613,  0.6078, -0.9464,  2.0103,  0.2073, -1.0235, -0.2593, -0.7512,
          0.0337,  0.9386],
        [ 0.4640, -0.0389, -0.5273,  0.0188, -1.1344,  0.5910,  1.2540,  0.5233,
         -1.7280,  1.7434],
        [-0.6744, -

In [12]:
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):

    N, C, H, W = input_data.shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1

    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3)
    return col.reshape(12,15,15).astype('int')

#d=np.arange(0,45*60)
#d=d.reshape(1,1,45,60)
d=data.reshape(1,1,60,45)
d=im2col(d,15,15,stride=15)
d.shape

(12, 15, 15)

In [20]:
data.shape

(1, 12, 15, 15)

In [40]:
x=np.random.randn(1,2,8)
y=np.random.randn(2,2,8)
np.vstack([y,x]).shape

(3, 2, 8)

In [22]:
x.reshape(1,-1).shape

(1, 8)

In [36]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.fc = nn.Linear(288, 3)
        
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNN()

'''最適化手法の定義'''
criterion = nn.MSELoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

x=data[0][1].reshape(1,1,15,15)
x=model(torch.tensor(x).float())


torch.Size([1, 2])

In [1]:
import pygame
import sys, os
import random

import numpy as np
from copy import copy
from pygame.locals import *
from gridworld import grid_world
from policy import Inp, Crew




env=grid_world()
agent_pos1,agent_pos2,agent_pos3,agent_inp_pos,data=env.reset()

cr1=Crew()
cr2=Crew()
cr3=Crew()
inp=Inp()
action_inp=0
Start=False
i=0
while True:
    env.render()
    for event in pygame.event.get():
        if event.type == KEYDOWN:
            Start=True
    if Start==False:
        continue
    action1=cr1.forward(agent_pos1,agent_inp_pos,data)
    action2=cr2.forward(agent_pos2,agent_inp_pos,data)
    action3=cr3.forward(agent_pos3,agent_inp_pos,data)
    action_inp=inp.forward(agent_inp_pos,agent_pos1,agent_pos2,agent_pos3)
    agent_pos1,agent_pos2,agent_pos3,agent_inp_pos,done,data=env.step(action1,action2,action3,action_inp)

    """if i%30==0:
        print(action_inp)"""


    if done==True:
        agent_pos1,agent_pos2,agent_pos3,agent_inp_pos=env.reset()
        i=0
    i+=1




    pygame.display.flip()


pygame 2.0.0.dev6 (SDL 2.0.10, python 3.7.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/tataraishouta/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3296, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-2f404972b82d>", line 25, in <module>
    env.render()
  File "/Users/tataraishouta/Documents/game/gridworld.py", line 145, in render
    (chip_list[c],(self.grid_size, self.grid_size)))
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/tataraishouta/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2033, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/tataraishouta/anaconda3/lib/python3.7/site-packages/IPython/core/ultratb.py", line 1

KeyboardInterrupt: 

In [2]:
data.shape

(2700,)

In [29]:
import random

x=[random.randrange(1,10) for i in range(100)]
d=np.zeros(10)


In [34]:
for b in x:
    d[b]+=1
np.argmax(d)+1

5

In [40]:
random.randrange(1,5)

3

In [42]:
np.abs(-4)

4

In [43]:
list(range(0,4))

[0, 1, 2, 3]

In [46]:
a=[1,-1,0,0]


TypeError: '<' not supported between instances of 'int' and 'list'

In [51]:
x=['a','b','c']
x.pop(1)

'b'